In [ ]:
!pip install pmdarima==1.2.1
!wget https://launchpad.net/~mario-mariomedina/+archive/ubuntu/talib/+files/libta-lib0_0.4.0-oneiric1_amd64.deb -qO libta.deb
!wget https://launchpad.net/~mario-mariomedina/+archive/ubuntu/talib/+files/ta-lib0-dev_0.4.0-oneiric1_amd64.deb -qO ta.deb
!dpkg -i libta.deb ta.deb
!pip install TA-Lib==0.4.17

     |████████████████████████████████| 921kB 5.8MB/s 
     |████████████████████████████████| 24.8MB 63.2MB/s 
ERROR: umap-learn 0.4.6 has requirement scipy>=1.3.1, but you'll have scipy 1.2.3 which is incompatible.
ERROR: tensorflow 2.3.0 has requirement scipy==1.4.1, but you'll have scipy 1.2.3 which is incompatible.
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
  Found existing installation: scipy 1.4.1
    Uninstalling scipy-1.4.1:
      Successfully uninstalled scipy-1.4.1
Selecting previously unselected package libta-lib0.
(Reading database ... 144793 files and directories currently installed.)
Preparing to unpack libta.deb ...
Unpacking libta-lib0 (0.4.0-oneiric1) ...
Selecting previously unselected package ta-lib0-dev.
Preparing to unpack ta.deb ...
Unpacking ta-lib0-dev (0.4.0-oneiric1) ...
Setting up libta-lib0 (0.4.0-oneiric1) ...
Setting up ta-lib0-dev (0.4.0-oneiric1) ...
Processing triggers for man-

In [ ]:
#mounting for using dataset and saving work
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
from scipy.stats import kurtosis
from pmdarima import auto_arima
import pmdarima as pm
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense, LSTM
from keras.callbacks import EarlyStopping
import talib 
import json

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
 #percentile difference between actual and predicted
def mean_absolute_percentage_error(actual, prediction):
    actual = pd.Series(actual)
    prediction = pd.Series(prediction)
    #mathematical calculation
    return 100 * np.mean(np.abs((actual - prediction))/actual)

  #ARIMA application
def get_arima(data, train_len, test_len):
    # prepare train and test data
    data = data.tail(test_len + train_len).reset_index(drop=True)
    train = data.head(train_len).values.tolist()
    test = data.tail(test_len).values.tolist()

    # Initialize model
    #max_p is the max value of AR and max_q is the max value of MA
    #train training data
    model = auto_arima(train, max_p=3, max_q=3, seasonal=False, trace=True,
                       error_action='ignore', suppress_warnings=True)

    # Determine model parameters
    #fitting model by exact maximum likelihood
    model.fit(train)
    order = model.get_params()['order']
    print('ARIMA order:', order, '\n')

    # Genereate predictions
    prediction = []
    for i in range(len(test)):
        #testing ARIMA
        model = pm.ARIMA(order=order)
        model.fit(train)
        #printing the percentage of data completed while testing
        print('working on', i+1, 'of', test_len, '-- ' + str(int(100 * (i + 1) / test_len)) + '% complete')
        prediction.append(model.predict()[0])
        train.append(test[i])

    # Generate error data
      #mean squared error
    mse = mean_squared_error(test, prediction)
      #root mean square error
    rmse = mse ** 0.5
      #mean absolute percentage error
    mape = mean_absolute_percentage_error(pd.Series(test), pd.Series(prediction))
    return prediction, mse, rmse, mape

#
def get_lstm(data, train_len, test_len, lstm_len=4):
    # prepare train and test data
    data = data.tail(test_len + train_len).reset_index(drop=True)
      #reshaping the high volatality dataset 
    dataset = np.reshape(data.values, (len(data), 1))
      #transform features by scaling each feature to the given range
    scaler = MinMaxScaler(feature_range=(0, 1))
      #fitting model by maximum likelihood
    dataset_scaled = scaler.fit_transform(dataset)
      #empty list for input and output of train and test
    x_train = []
    y_train = []
    x_test = []
      #splitting up the train and test dataset and reshaping it
    for i in range(lstm_len, train_len):
        x_train.append(dataset_scaled[i - lstm_len:i, 0])
        y_train.append(dataset_scaled[i, 0])
    for i in range(train_len, len(dataset_scaled)):
        x_test.append(dataset_scaled[i - lstm_len:i, 0])

    x_train = np.array(x_train)
    y_train = np.array(y_train)
    x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))
    x_test = np.array(x_test)
    x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], 1))

    # Set up & fit LSTM RNN (3 layers)
    model = Sequential()
    model.add(LSTM(units=lstm_len, return_sequences=True, input_shape=(x_train.shape[1], 1)))
    model.add(LSTM(units=int(lstm_len/2)))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='mean_squared_error', optimizer='adam')
    early_stopping = EarlyStopping(monitor='loss', mode='min', verbose=1, patience=5)
    model.fit(x_train, y_train, epochs=50, batch_size=1, verbose=2, callbacks=[early_stopping])

    # Generate predictions
    prediction = model.predict(x_test)
    prediction = scaler.inverse_transform(prediction).tolist()
      #storing predictions
    output = []
    for i in range(len(prediction)):
        output.extend(prediction[i])
    prediction = output

    # Generate error data
      #mean squared error
    mse = mean_squared_error(data.tail(len(prediction)).values, prediction)
      #root mean square error
    rmse = mse ** 0.5
      #mean absolute percentage error
    mape = mean_absolute_percentage_error(data.tail(len(prediction)).reset_index(drop=True), pd.Series(prediction))
    return prediction, mse, rmse, mape


In [ ]:
if __name__ == '__main__':
      # Loading data
    path = 'drive/My Drive/data/'+'BA'+'_2006-01-01_to_2020-09-30.csv'
    data1 = pd.read_csv(path,header=None)
      # changing column order to: ['Date', 'Open', 'High', 'Low', 'Close', 'Volume','Adj Close']
    new_col=[0,3,1,2,4,5,6,7]
      #assigning changed order 
    set(data1.columns) == set(new_col)
    data=data1[new_col]
      #dropping headers of the columns with names(str)
    data = data.drop([0], axis=0)
      #Initialize moving averages from Ta-Lib, store functions in dictionary
    talib_moving_averages = ['SMA', 'EMA', 'WMA', 'DEMA', 'KAMA', 'MIDPOINT', 'T3', 'TEMA', 'TRIMA']

    l=len(talib_moving_averages)
      # Determine kurtosis "K" values for MA period 4-99
    kurtosis_results = {'period': []} #dictionary to determine time period for MA
    for i in range(4,100):
        kurtosis_results['period'].append(i)
        for ma in range(l):
            # Run moving average, removing last 252 days (used later for test data set), trimming MA result to last 60 days
            if ma == 0:
              ma_output = talib.SMA(data.iloc[1006:3712,4],i).tail(60) #inbuilt function of TA-Lib syntax: talib._MA(data,timeperiod)
            elif ma == 1:
              ma_output = talib.EMA(data.iloc[1006:3712,4],i).tail(60)
            elif ma == 2:
              ma_output = talib.WMA(data.iloc[1006:3712,4],i).tail(60)
            elif ma == 3:
              ma_output = talib.DEMA(data.iloc[1006:3712,4],i).tail(60)
            elif ma == 4:
              ma_output = talib.KAMA(data.iloc[1006:3712,4],i).tail(60)
            elif ma == 5:
              ma_output = talib.MIDPOINT(data.iloc[1006:3712,4],i).tail(60)
            elif ma == 6:
              ma_output = talib.T3(data.iloc[1006:3712,4],i).tail(60)
            elif ma == 7:
              ma_output = talib.TEMA(data.iloc[1006:3712,4],i).tail(60)
            elif ma == 8:
              ma_output = talib.TRIMA(data.iloc[1006:3712,4],i).tail(60)

            # Determine kurtosis "K" value
            k = kurtosis(ma_output, fisher=False)
    
            # add to dictionary
            if ma not in kurtosis_results.keys():
                kurtosis_results[ma] = []
            kurtosis_results[ma].append(k)

    kurtosis_results = pd.DataFrame(kurtosis_results)
    #print(kurtosis_results)

    # Determine period with K closest to 3 +/- 5% (mesokurtic) 
    # Ordering the time period for the mesokurtic MA's in optimized_period
    optimized_period = {}
    for ma in range(l):
        difference = np.abs(kurtosis_results[ma] - 3)
        df = pd.DataFrame({'difference': difference, 'period': kurtosis_results['period']})
        df = df.sort_values(by=['difference'], ascending=True).reset_index(drop=True)

         # ordering and considering only mesokurtic time periods from kurtosis_results
        if df.at[0, 'difference'] < 3 * 0.05:
            optimized_period[ma] = df.at[0, 'period']
        else:
            print(str(ma) + ' is not viable, best K greater or less than 3 +/- 5%')

    print('\nOptimized periods:', optimized_period)

    simulation = {}
    for ma in optimized_period:
        # Split data into low volatility and high volatility time series for ARIMA and LSTM based on MA type
        if ma == 0:
          low_vol = talib.SMA(data[4],optimized_period[ma]) #inbuilt function of TA-Lib
          high_vol = data[4] - low_vol                      #subtracting 'Close' attribute from low voltalie dataset
        elif ma == 1:
          low_vol = talib.EMA(data[4],optimized_period[ma])
          high_vol = data[4] - low_vol
        elif ma == 2:
          low_vol = talib.WMA(data[4],optimized_period[ma])
          high_vol = data[4] - low_vol
        elif ma == 3:
          low_vol = talib.DEMA(data[4],optimized_period[ma])
          high_vol = data[4] - low_vol
        elif ma == 4:
          low_vol = talib.KAMA(data[4],optimized_period[ma])
          high_vol = data[4] - low_vol
        elif ma == 5:
          low_vol = talib.MIDPOINT(data[4],optimized_period[ma])
          high_vol = data[4] - low_vol
        elif ma == 6:
          low_vol = talib.T3(data[4],optimized_period[ma])
          high_vol = data[4] - low_vol
        elif ma == 7:
          low_vol = talib.TEMA(data[4],optimized_period[ma])
          high_vol = data[4] - low_vol
        elif ma == 8:
          low_vol = talib.TRIMA(data[4],optimized_period[ma])
          high_vol = data[4] - low_vol

        # Generate ARIMA and LSTM predictions
        print('\nWorking on ' + str(ma) + ' predictions')
        #compatibility of MA in ARIMA for dataset by calling ARIMA 
        try:
            low_vol_prediction, low_vol_mse, low_vol_rmse, low_vol_mape = get_arima(low_vol, 1000, 252)
        except:
            print('ARIMA error, skipping to next MA type')
            continue
        #calling LSTM with high volatility parameters
        high_vol_prediction, high_vol_mse, high_vol_rmse, high_vol_mape = get_lstm(high_vol, 1000, 252)

        #calculating mse,rmse and mape to differentiate predictions and original
        final_prediction = pd.Series(low_vol_prediction) + pd.Series(high_vol_prediction)
        mse = mean_squared_error(final_prediction.values, data[4].tail(252).values)
        rmse = mse ** 0.5
        mape = mean_absolute_percentage_error(data[4].tail(252).reset_index(drop=True), final_prediction)

        # Generate prediction accuracy
        actual = data[4].tail(252).values
        result_1 = []
        result_2 = []
        for i in range(1, len(final_prediction)):
            # Compare prediction to previous close price
            if final_prediction[i] > actual[i-1] and actual[i] > actual[i-1]:
                result_1.append(1)
            elif final_prediction[i] < actual[i-1] and actual[i] < actual[i-1]:
                result_1.append(1)
            else:
                result_1.append(0)

            # Compare prediction to previous prediction
            if final_prediction[i] > final_prediction[i-1] and actual[i] > actual[i-1]:
                result_2.append(1)
            elif final_prediction[i] < final_prediction[i-1] and actual[i] < actual[i-1]:
                result_2.append(1)
            else:
                result_2.append(0)

        accuracy_1 = np.mean(result_1)
        accuracy_2 = np.mean(result_2)
          #storing predicted and the actual value
        simulation[ma] = {'low_vol': {'prediction': low_vol_prediction, 'mse': low_vol_mse,
                                      'rmse': low_vol_rmse, 'mape': low_vol_mape},
                          'high_vol': {'prediction': high_vol_prediction, 'mse': high_vol_mse,
                                       'rmse': high_vol_rmse},
                          'final': {'prediction': final_prediction.values.tolist(), 'mse': mse,
                                    'rmse': rmse, 'mape': mape},
                          'accuracy': {'prediction vs close': accuracy_1, 'prediction vs prediction': accuracy_2}}

        # save simulation data here as checkpoint
        with open('drive/My Drive/data/results/simulation_data.json', 'w') as fp:
            json.dump(simulation, fp)

In [ ]:
#comparing prediction vs original dataset
for ma in simulation.keys():
        print('\n' + str(ma))
        print('Prediction vs Close:\t\t' + str(round(100*simulation[ma]['accuracy']['prediction vs close'], 2))
              + '% Accuracy')
        print('Prediction vs Prediction:\t' + str(round(100*simulation[ma]['accuracy']['prediction vs prediction'], 2))
              + '% Accuracy')
        print('MSE:\t', simulation[ma]['final']['mse'],
              '\nRMSE:\t', simulation[ma]['final']['rmse'],
              '\nMAPE:\t', simulation[ma]['final']['mape'])


1
Prediction vs Close:		51.0% Accuracy
Prediction vs Prediction:	46.22% Accuracy
MSE:	 323.5499349640634 
RMSE:	 17.987493848895777 
MAPE:	 5.490833127018244

2
Prediction vs Close:		52.99% Accuracy
Prediction vs Prediction:	47.81% Accuracy
MSE:	 659.8241498287745 
RMSE:	 25.687042450013088 
MAPE:	 8.411223935964577

3
Prediction vs Close:		50.2% Accuracy
Prediction vs Prediction:	47.01% Accuracy
MSE:	 162.87786625652433 
RMSE:	 12.76236131194084 
MAPE:	 4.206384989658252

4
Prediction vs Close:		50.6% Accuracy
Prediction vs Prediction:	46.61% Accuracy
MSE:	 316.95786750860594 
RMSE:	 17.803310577210237 
MAPE:	 5.238793308460417

5
Prediction vs Close:		52.59% Accuracy
Prediction vs Prediction:	48.61% Accuracy
MSE:	 274.3950291643885 
RMSE:	 16.564873351897035 
MAPE:	 6.367435051312706

8
Prediction vs Close:		45.82% Accuracy
Prediction vs Prediction:	46.61% Accuracy
MSE:	 987.4568844423396 
RMSE:	 31.4238266995339 
MAPE:	 10.651317134884952
